In [1]:
%cd /opt

/opt


In [2]:
%%capture
!tar xvf /kaggle/input/extract-prebuilt-kaldi-from-docker/kaldi.tar

In [3]:
%cd /tmp

/tmp


In [4]:
!git clone https://github.com/pytorch/fairseq/

Cloning into 'fairseq'...
remote: Enumerating objects: 28172, done.
remote: Counting objects: 100% (586/586), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 28172 (delta 308), reused 464 (delta 236), pack-reused 27586
Receiving objects: 100% (28172/28172), 11.87 MiB | 16.19 MiB/s, done.
Resolving deltas: 100% (21131/21131), done.


In [5]:
%%capture
!pip install phonemizer

In [6]:
%%capture
!pip install git+https://github.com/pytorch/fairseq/

In [7]:
%%capture
!apt-get -y install espeak

In [8]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 11.43 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [9]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [10]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [11]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [12]:
!cat /kaggle/input/wav2vec-u-cv-swedish-audio/*.wrd | grep -v '^$' | sort| uniq > /kaggle/working/sentences.txt


In [13]:
%cd fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [14]:
%%capture
!apt-get -y install zsh

In [15]:
!mkdir /kaggle/working/preppedtext

In [16]:
%cd scripts

/tmp/fairseq/examples/wav2vec/unsupervised/scripts


The next part requires a FastText language id model; I don't know where the 187 language model comes from, but there is a model for 176 languages [here](https://fasttext.cc/docs/en/language-identification.html#content)

In [17]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2021-05-30 14:04:44--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin’

lid.176.bin         100%[===================>] 125.18M  47.5MB/s    in 2.6s    

2021-05-30 14:04:47 (47.5 MB/s) - ‘lid.176.bin’ saved [131266198/131266198]



In [18]:
!cat normalize_and_filter_text.py|sed -e 's/187/176/' > tmp
!mv tmp normalize_and_filter_text.py

In [19]:
# Needed to see what's going wrong
os.environ['HYDRA_FULL_ERROR'] = '1'

In [20]:
!mkdir /tmp/fairseq/examples/speech_recognition/kaldi/config

There are two lines with missing variables in `prepare_text.sh` - [pull request](https://github.com/pytorch/fairseq/pull/3569) - so replace the file.

While I'm replacing the file: most of the first part of the script is unneeded, as I already have a phonetic dictionary, so I'm use that instead.

With the calls of the `preprocess.py` script, make sure to check the threshold: there's a divide by zero if the threshold is set too high.

In [21]:
%%writefile prepare_text.sh
#!/usr/bin/env zsh
# Copyright (c) Facebook, Inc. and its affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

lg=$1
text_path=$2
target_dir=$3

#ph_lg=${lg:l}
#if test "$lg" = 'fr'; then
#  ph_lg='fr-fr'
#elif test "$lg" = 'en'; then
#  ph_lg='en-us'
#elif test "$lg" = 'pt'; then
#  ph_lg='pt-br'
#fi
ph_lg="sv"

echo $lg
echo $ph_lg
echo $text_path
echo $target_dir

mkdir -p $target_dir
#python normalize_and_filter_text.py --lang $lg < $text_path | grep -v '\-\-\-' >! $target_dir/lm.upper.lid.txt
#python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/lm.upper.lid.txt --only-source --destdir $target_dir --thresholdsrc 2 --padding-factor 1 --dict-only
#cut -f1 -d' ' $target_dir/dict.txt | grep -v -x '[[:punct:]]*' | grep -Pv '\d\d\d\d\d+' >! $target_dir/words.txt
cp /kaggle/input/wav2vec-u-cv-swedish-audio/train.wrd $target_dir/lm.upper.lid.txt
cut -f1 -d' ' /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train >! $target_dir/words.txt

#one=$(echo "1" | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -p ' ' -w '' -l $ph_lg --language-switch remove-flags)
#sed 's/$/ 1/' $target_dir/words.txt | PHONEMIZER_ESPEAK_PATH=$(which espeak) phonemize -o $target_dir/phones.txt -p ' ' -w '' -l $ph_lg -j 70 --language-switch remove-flags
cut -f2- -d' ' /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train >! $target_dir/phones.txt

#echo "one is ${one}"

#sed -i "s/${one}$//" $target_dir/phones.txt
#paste $target_dir/words.txt $target_dir/phones.txt >! $target_dir/lexicon.lst
cp /kaggle/input/wav2vec-u-cv-swedish-audio/dict.train $target_dir/lexicon.lst

#python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones.txt --only-source --destdir $target_dir/phones --thresholdsrc 1000 --padding-factor 1 --dict-only
python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones.txt --only-source --destdir $target_dir/phones --thresholdsrc 2 --padding-factor 1 --dict-only

python filter_lexicon.py -d $target_dir/phones/dict.txt < $target_dir/lexicon.lst >! $target_dir/lexicon_filtered.lst
python phonemize_with_sil.py -s 0.25 --surround --lexicon $target_dir/lexicon_filtered.lst < $target_dir/lm.upper.lid.txt >! $target_dir/phones/lm.phones.filtered.txt
cp $target_dir/phones/dict.txt $target_dir/phones/dict.phn.txt
echo "<SIL> 0" >> $target_dir/phones/dict.phn.txt
python $FAIRSEQ_ROOT/fairseq_cli/preprocess.py --dataset-impl mmap --trainpref $target_dir/phones/lm.phones.filtered.txt --workers 70 --only-source --destdir $target_dir/phones --srcdict $target_dir/phones/dict.phn.txt

lmplz -o 4 < $target_dir/lm.upper.lid.txt --discount_fallback --prune 0 0 0 3 >! $target_dir/kenlm.wrd.o40003.arpa
build_binary $target_dir/kenlm.wrd.o40003.arpa $target_dir/kenlm.wrd.o40003.bin
lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_words_sil lm_arpa=$target_dir/kenlm.wrd.o40003.arpa wav2letter_lexicon=$target_dir/lexicon_filtered.lst data_dir=$target_dir/phones "blank_symbol='<SIL>'" "in_labels='phn_to_words_sil'"
lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_words lm_arpa=$target_dir/kenlm.wrd.o40003.arpa wav2letter_lexicon=$target_dir/lexicon_filtered.lst data_dir=$target_dir/phones  "in_labels='phn_to_words'"

lmplz -o 4 < $target_dir/phones/lm.phones.filtered.txt --discount_fallback >! $target_dir/phones/lm.phones.filtered.04.arpa
build_binary -s $target_dir/phones/lm.phones.filtered.04.arpa $target_dir/phones/lm.phones.filtered.04.bin
lmplz -o 6 < $target_dir/phones/lm.phones.filtered.txt --discount_fallback >! $target_dir/phones/lm.phones.filtered.06.arpa
build_binary -s $target_dir/phones/lm.phones.filtered.06.arpa $target_dir/phones/lm.phones.filtered.06.bin

lg=$lg python $FAIRSEQ_ROOT/examples/speech_recognition/kaldi/kaldi_initializer.py fst_dir=$target_dir/fst/phn_to_phn_sil lm_arpa=$target_dir/phones/lm.phones.filtered.06.arpa data_dir=$target_dir/phones "blank_symbol='<SIL>'" "in_labels='phn_to_phn_sil'"

Overwriting prepare_text.sh


`config.yaml` needs to exist. Most of the options are set on the command line, but not everything: `in_labels`, `out_labels`, `kaldi_root` and (for one run) `silence_symbol`. The options are:

- `in_labels`
- `wav2letter_lexicon`: path to wav2letter lexicon: set on the command line
- `out_labels`
- `kaldi_root`: path to Kaldi: `/opt/kaldi` for my kaggle image
- `fst_dir`: path where generated fsts will be saved: set on the command line
- `data_dir`: path to phones data: set on the command line
- `lm_arpa`: path to the lm in ARPA format: set on the command line
- `blank_symbol`: CTC blank symbol (`<s>` is used in the script)
- `silence_symbol`: Kaldi symbol for silence (`<SIL>` is set on the command line for two of the scripts)



In [22]:
%%writefile /tmp/fairseq/examples/speech_recognition/kaldi/config/config.yaml
kaldi_root: "/opt/kaldi"

Writing /tmp/fairseq/examples/speech_recognition/kaldi/config/config.yaml


In [23]:
!zsh prepare_text.sh sv /kaggle/working/sentences.txt /kaggle/working/preppedtext

sv
sv
/kaggle/working/sentences.txt
/kaggle/working/preppedtext
=== 1/5 Counting and sorting n-grams ===
Reading /kaggle/working/preppedtext/lm.upper.lid.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 14359 types 3160
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:37920 2:2571430400 3:4821431808 4:7714291200
Statistics:
1 3160 D1=0.722623 D2=1.14413 D3+=1.45956
2 10285 D1=0.848104 D2=1.2466 D3+=1.46191
3 12632 D1=0.943362 D2=1.24166 D3+=1.32723
4 19/11699 D1=0.970399 D2=1.4843 D3+=2.12351
Memory estimate for binary LM:
type     kB
probing 617 assuming -p 1.5
probing 764 assuming -r models -p 1.5
trie    309 without quantization
trie    182 assuming -q 8 -b 8 quantization 
trie    293 assuming -a 22 array pointer compression
trie    166 assuming -a 22 -q 8 -b 8 array pointer compression and quant